In [1]:
from bs4 import BeautifulSoup
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import matplotlib.pyplot as plt
import pandas as pd

In [5]:
link = "https://data-class-mars-challenge.s3.amazonaws.com/Mars/index.html"

executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)
browser.visit(link)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [6]:
rows = soup.find_all("td", class_="data-row")
temp_data_lst = []

for row in rows:
    temp_dict = {}
    temp_dict["id"] = int(row.find("td").text)
    temp_dict["terrestrial_date"] = dt.datetime.strptime(row.find_next("td").find_next("td").text,"%Y-%m-%d")
    temp_dict["sol"] = int(row.find_next("td").find_next("td").find_next("td").text)
    temp_dict["ls"] = int(row.find_next("td").find_next("td").find_next("td").find_next("td").text)
    temp_dict["month"] = int(row.find_next("td").find_next("td").find_next("td").find_next("td").find_next("td").text)
    temp_dict["min_temp"] = float(row.find_next("td").find_next("td").find_next("td").find_next("td").find_next("td").find_next("td").text)
    temp_dict["pressure"] = float(row.find_next("td").find_next("td").find_next("td").find_next("td").find_next("td").find_next("td").find_next("td").text)
    temp_data_lst.append(temp_dict)

In [7]:
mars_temp_df = pd.DataFrame(temp_data_lst)
mars_temp_df

""


In [13]:
mars_temp_df.describe()

In [14]:
mnth = mars_temp_df["month"].max()
print(f"There are {mnth} month on Mars.")
# I'm assuming 'ls' is the martian date
days_on_mars = mars_temp_df["ls"].count()
print(f"A total of {days_on_mars} Martian days have been collected.")

In [15]:
temp_by_month = mars_temp_df[["month","min_temp"]]
temp_by_month = temp_by_month['min_temp'].groupby(mars_temp_df["month"]).mean()
temp_by_month_df = pd.DataFrame(temp_by_month)
temp_by_month_df

plt.bar([1,2,3,4,5,6,7,8,9,10,11,12], temp_by_month_df["min_temp"])
plt.title("Mars Average Temperature by Months")
plt.xlabel("Martian Months")

In [ ]:
print("The third Martian month is the coldest.")
print("The eigth Martian month is the warmest.")

In [ ]:
atmo_prssr = mars_temp_df[["month","pressure"]]
atmo_prssr_mnth_avg = atmo_prssr["pressure"].groupby(atmo_prssr["month"]).mean()
atmo_prssr_mnth_avg_df = pd.DataFrame(atmo_prssr_mnth_avg)
atmo_prssr_mnth_avg_df

plt.bar([1,2,3,4,5,6,7,8,9,10,11,12], atmo_prssr_mnth_avg_df["pressure"])

In [ ]:
print("The sixth month has the lowest atmospheric pressure.")
print("The ninth month has the highest atmospheric pressure.")

In [ ]:
# solution found to get one earth year source:
# https://stackoverflow.com/questions/29370057/select-dataframe-rows-between-two-dates

combined = mars_temp_df[["terrestrial_date","month","min_temp"]]

# define a date period for the martian days
# which is the 12th month of the 359th martian day
martian_year = (combined["terrestrial_date"] > "2013-8-1") & (combined["terrestrial_date"] <= "2015-6-18")
martian_days_in_year = combined.loc[martian_year]

# define  2013-08-02 plus 365 days
earth_year = (combined["terrestrial_date"] >= "2013-8-2") & (combined["terrestrial_date"] <= "2014-8-1")
earth_days_in_year = combined.loc[earth_year]

plt.figure(figsize=(10,5))
plt.plot(martian_days_in_year['terrestrial_date'], martian_days_in_year['min_temp'])

In [ ]:
plt.plot(earth_days_in_year['terrestrial_date'], earth_days_in_year['min_temp'])

In [ ]:
print("Disclosure: a few days are missing in the terrestrial_days column.")
print("An example would be from 2013-08-03 jumps to 2013-08-17")
earth_days_len = len(earth_days_in_year)
martian_days_len = len(martian_days_in_year)
print(f"This means that there are a total of {earth_days_len} Earth days in a year.")
print(f"This also means that there are a total of {martian_days_len} Martian days in a year.")
print()
difference = martian_days_len - earth_days_len
print(f"Mars has {difference} days more than Earth.")

In [ ]:
# Export to CSV.

mars_temp_df.to_csv("mars_temp.csv", index=False)